<a href="https://colab.research.google.com/github/vifirsanova/compling/blob/main/practice/tg_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

```2025 (c) В.И. Фирсанова```

### **Что такое асинхронность**  
Обычные программы выполняют задачи **по очереди**. Например, вы написали программу, которая состоит из двух шагов: загрузки данных и вывода сообщения на экран. В каком порядке будут выводиться сообщения? Сообщение появится только после завершения загрузки. Такой тип программ не подойдет для создания бота: пользователь будет ждать выполнения каждого действия очень долго. Это ухудшает пользовательский опыт.

Асинхронные программы работают **параллельно**. Это значит, что программа может переключиться на другую задачу, не останавливая текущую. Например, мы сможем выводить сообщения для пользователя в процессе загрузки данных.  

### **Почему Telegram-боты делают асинхронными**  
**Асинхронный бот**:  
- обрабатывает несколько сообщений одновременно
- отправляет запросы на сервер и продолжает работу, не дожидаясь ответа сервера

Телеграм позволяет разрабатывать **синхронные** боты. Такой вариант больше подходит для новичков, но такие боты:
- отвечают медленнее при большом количестве запросов
- ждут завершения каждой операции перед обработкой следующего сообщения

### **Что такое `asyncio`, `async`, `await`**  
Это методы, которые мы освоим на сегодняшнем туториале
- `asyncio` — библиотека для работы с асинхронным кодом
- `async` — объявляет асинхронную функцию
- `await` — приостанавливает выполнение, позволяя другим задачам выполняться в это время

Где изучать библиотеки для создания Telegram-ботов?

`aiogram`
- асинхронная библиотека
- использует `asyncio`
- одна из самых производительных

*Учим здесь: https://mastergroosha.github.io/aiogram-3-guide/*

`telebot`
- синхронный бот
- подходит для первых экспериментов

*Образец использования: https://habr.com/ru/articles/580408/*

Другие библиотеки: `pyrogram`, `telethon`, `python-telegram-bot`

Советы:
- не бойтесь разбираться в официальной документации
- обращайте внимание на версии библиотек (например, `aiogram 2` и `aiogram 3` очень разные!)
- Хабр в помощь! Там очень много ценных материалов по разработке ТГ-ботов

### Работа с `aiogram` и знакомство с асинхронным кодом

In [1]:
!pip install aiogram -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 610.0/610.0 kB 10.1 MB/s eta 0:00:00


In [2]:
# Импортируем необходимые модули
from aiogram import Bot, Dispatcher, types  # Основные классы для работы с ботом
import logging  # Логирование для отслеживания работы бота
import asyncio  # Модуль для работы с асинхронным кодом
import sys  # Используется для работы с системными вызовами

# Токен API бота (его нужно заменить на реальный токен, полученный у BotFather)
API_TOKEN = ""

# Настраиваем логирование, чтобы видеть информацию о работе бота в консоли
logging.basicConfig(level=logging.INFO)
# Создаем объект диспетчера, который управляет входящими сообщениями и командами
dp = Dispatcher()

# Декоратор @dp.message() указывает, что функция будет обрабатывать входящие сообщения
@dp.message()
async def echo(message: types.Message):
    """
    Асинхронная функция (корутина), которая отвечает пользователю.
    Она получает объект сообщения и отправляет ответ.

    :param message: объект сообщения от пользователя
    """
    await message.answer("Привет! Я твой бот.")  # Отправляем ответное сообщение

async def main():
    """
    Основная асинхронная функция для запуска бота.
    1. Создает объект бота с API токеном.
    2. Запускает диспетчер, который начинает обрабатывать сообщения.
    """
    bot = Bot(token=API_TOKEN)
    await dp.start_polling(bot)

# Проверяем, запущен ли скрипт напрямую (не импортирован в другой файл)
if __name__ == "__main__":
    # Запускаем основную функцию
    await main()

**Асинхронные функции (`async def`)**  
   - позволяет выполнять код без блокировки программы
   - например, ожидание ответа от сервера не мешает обработке новых сообщений
   - `await` используется внутри асинхронных функций для вызова других асинхронных операций, например, для ответа пользователю

**Объект `Dispatcher`**  
   - управляет обработкой входящих сообщений
   - декораторы (`@dp.message()`) регистрируют функции бота

**Объект `Bot`**  
   - используется для отправки сообщений и работы с API Telegram

In [8]:
from aiogram.filters import Command

dp = Dispatcher()

# Обрабатываем команды "/start" и "/help"
@dp.message(Command("start", "help")) # Фильтр команд
async def send_welcome(message: types.Message):
    """
    Функция отвечает пользователю, когда он отправляет команду /start или /help.
    """
    await message.answer("Привет! Я бот, который может отвечать на частые вопросы.")

# Создаём словарь с ответами на часто задаваемые вопросы (FAQ)
faq = {
    "цена": "Цена зависит от тарифа. Посетите наш сайт.",
    "время работы": "Мы работаем с 9:00 до 18:00 без выходных."
}

# Обрабатываем все входящие сообщения
@dp.message()
async def answer_faq(message: types.Message):
    """
    Функция получает текст сообщения, проверяет его в словаре и отправляет ответ.
    Если вопрос не найден в словаре, бот отправляет сообщение о неизвестном вопросе.
    """
    text = message.text.lower() # Приводим текст к нижнему регистру
    response = faq.get(text, "Я не знаю ответа на этот вопрос.") # Ищем ответ в словаре
    await message.answer(response)  # Отправляем ответ пользователю

async def main():
    bot = Bot(token=API_TOKEN)
    await dp.start_polling(bot)

if __name__ == "__main__":
    await main()

Мы научились обрабатывать команды и работать со словарем

In [13]:
dp = Dispatcher()

@dp.message(Command("start", "help"))
async def send_welcome(message: types.Message):
    await message.answer("Привет! Я бот, который может парсить документы.")

# Декоратор указывает, что функция `handle_docs` должна быть вызвана,
# когда пользователь отправляет сообщение, содержащее документ.
@dp.message(lambda message: message.document)
async def handle_docs(message: types.Message):
    # Асинхронная функция, которая обрабатывает полученное сообщение
    document = message.document
    # Извлекаем информацию о документе из сообщения
    file_info = f"Файл: {document.file_name}\nРазмер: {document.file_size} байт" # Создаем строку с информацией о файле (имя и размер)
    await message.answer(file_info) # Отправляем пользователю сообщение с информацией о файле

async def main():
    bot = Bot(token=API_TOKEN)
    await dp.start_polling(bot)

if __name__ == "__main__":
    await main()

Мы научились принимать документы через бот

In [15]:
import aiofiles

dp = Dispatcher()
bot = Bot(token=API_TOKEN)

@dp.message(Command("start", "help"))
async def send_welcome(message: types.Message):
    await message.answer("Привет! Я бот, который может парсить документы.")

# Обрабатываем входящие документы (файлы)
@dp.message(lambda message: message.document) # Проверяем, является ли сообщение документом
async def handle_text_file(message: types.Message):
    """
    Функция загружает отправленный пользователем текстовый файл,
    читает его содержимое и считает количество слов.
    """
    # Скачиваем файл во временную директорию
    await bot.download(
        message.document, # Наш файл
        destination="temp.txt" # Имя файла, под которым он будет сохранён
    )

    # Асинхронно открываем файл и читаем его содержимое
    async with aiofiles.open("temp.txt", mode='r', encoding='utf-8') as f:
        content = await f.read()

    # Считаем количество слов в тексте
    word_count = len(content.split())

    # Отправляем пользователю информацию о количестве слов
    await message.answer(f"Файл загружен. Количество слов: {word_count}")

async def main():
    await dp.start_polling(bot)

if __name__ == "__main__":
    await main()

Теперь мы умеем обрабатывать документы

In [11]:
from aiogram.types import ReplyKeyboardMarkup, KeyboardButton

dp = Dispatcher()

# Создаём список кнопок для клавиатуры
kb = [
    [
        KeyboardButton(text="О боте"), # Кнопка для запроса информации о боте
        KeyboardButton(text="Помощь")  # Кнопка для получения справки
    ]
]

# Создаём объект клавиатуры с кнопками
keyboard = ReplyKeyboardMarkup(
    keyboard=kb, # Передаём список кнопок
    resize_keyboard=True, # Уменьшаем клавиатуру под размер экрана
    input_field_placeholder="Выберите действие" # Текст-подсказка в поле ввода
    )

@dp.message(Command("start"))
async def start_command(message: types.Message):
    await message.answer("С чем вам помочь?", reply_markup=keyboard) # Отправляем сообщение с клавиатурой в команду start

# Обрабатываем нажатие кнопки "О боте"
@dp.message(lambda message: message.text == "О боте") # Фильтр для сообщений с текстом "О боте"
async def about_bot(message: types.Message):
    """
    Функция отвечает пользователю, если он нажал кнопку "О боте".
    """
    await message.answer("Я Telegram-бот, который помогает обрабатывать текст и файлы!")

# Обрабатываем нажатие кнопки "Помощь"
@dp.message(lambda message: message.text == "Помощь") # Фильтр для сообщений с текстом "Помощь"
async def about_bot(message: types.Message):
    """
    Функция отправляет пользователю справочную информацию.
    """
    await message.answer("Привет! Я бот, который может парсить документы.")

async def main():
    bot = Bot(token=API_TOKEN)
    await dp.start_polling(bot)

if __name__ == "__main__":
    await main()

И теперь мы можем создавать кнопки с разным функционалом